# Reinforcement Learning
# Cours 3 : Policy Iteration and Value Iteration

Pour trouver une politique optimale, il existe deux grandes familles d'algorithmes : la programmation dynamique (résoudre le problème en le décomposant récursivement en plus petits problèmes) et les simulations de Monte-Carlo (faire des expériences pour estimer les distributions de probabilités). 

Dans ce TP, nous étudions deux types d'algorithme utilisant la programmation dynamique : les itérations sur les valeurs et les itérations sur la politique.


RAPPEL : 1/4 de la note finale est liée à la mise en forme : 

* pensez à nettoyer les outputs inutiles (installation, messages de débuggage, ...)
* soignez vos figures : les axes sont-ils faciles à comprendre ? L'échelle est adaptée ? 
* commentez vos résultats : vous attendiez-vous à les avoir ? Est-ce étonnant ? Faites le lien avec la théorie.

Ce TP reprend l'exemple d'un médecin et de ses vaccins. Vous allez comparer plusieurs stratégies et trouver celle optimale.
Un TP se fait en groupe de 2 à 4. Aucun groupe de plus de 4 personnes. 

Vous allez rendre le TP dans une archive ZIP. L'archive ZIP contient ce notebook au format `ipynb`, mais aussi exporté en PDF & HTML. 
L'archive ZIP doit aussi contenir un fichier txt appelé `groupe.txt` sous le format:

```
Nom1, Prenom1, Email1, NumEtudiant1
Nom2, Prenom2, Email2, NumEtudiant2
Nom3, Prenom3, Email3, NumEtudiant3
Nom4, Prenom4, Email4, NumEtudiant4
```

Un script vient extraire vos réponses : ne changez pas l'ordre des cellules et soyez sûrs que les graphes sont bien présents dans la version notebook soumise. 

In [1]:
import matplotlib.pyplot as plt
import torch
import networkx as nx

/home/unikarah/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### I. Estimation de la fonction de valeur d'un gridword

Nous avons vu en cours que :

$$v_\pi (s) = \mathbb{E}_\pi \left( G_t | s \right) = \sum_{s'} p(s'|s, a)\left[r+\gamma v_\pi(s') \right]$$

Dans le cas où les dynamiques de l'environnement sont entièrement connus, $p(s'|s, a)$ peut s'exprimer sous la forme d'un tensor et l'équation précédente aboutit à un système d'équations linéaires. Le problème est donc résolvable, mais la résolution risque d'être longue si l'environnement est grand. 

On cherche plutôt une résolution itérative qui applique le principe de la programmation dynamique. Concrètement, on part d'une fonction de valeur arbitraire $v_0$ (par exemple nulle partout), puis on y applique à chaque étape l'équation de Bellman :
$$v_{k+1} (s) = \sum_{s'} p(s'|s, a)\left[r+\gamma v_k(s') \right]$$
Lorsque l'algorithme a convergé vers un point fixe $v_\infty$, nous avons fini d'évaluer $v_\pi$, puisque ce dernier est l'unique point fixe de la fonction de valeur.

Cet algorithme est appelé l'**évaluation itérative de la politique**.

On considère par la suite le "gridworld" suivant :

![gridworld](img/grid-world.png)

Les cases grisées sont terminales et la récompense est de -1 sur toutes les transitions.
La taille du gridworld est une constante `CUBE_SIDE`.

**Q1: évaluez la fonction de valeur de la politique aléatoire à l'aide d'un algorithme itératif. Arrếtez l'algorithme lorsque les valeurs n'ont pas évolué de plus de 1e-2.**

In [45]:
from tabulate import tabulate
import typing as t
from dataclasses import dataclass, field
import random
import torch
import matplotlib.pyplot as plt

Action = t.Literal["L", "R", "U" , "D"]
CUBE_SIDE = 6

@dataclass
class State: 
    """
    It represents any cell in the world
    """
    cell: int
    value: int = 0
    
    def __post_init__(self):
        self.bounds = {
            'L': self.cell - self.cell % CUBE_SIDE,
            'R': self.cell - self.cell % CUBE_SIDE + (CUBE_SIDE - 1),
            'U': self.cell % CUBE_SIDE,
            'D': self.cell % CUBE_SIDE + CUBE_SIDE * (CUBE_SIDE - 1),
        }
        self.neighbors = [self.act(a) for a in "LRUD"]
        assert all(i >= 0 and i < CUBE_SIDE*CUBE_SIDE for i in self.neighbors)
    
    def is_termination(self):
        return self.cell in {0, CUBE_SIDE * CUBE_SIDE - 1}

    def act(self, a: Action):
        """
        Get next state
        """
        if a == 'L': 
            return min(self.bounds['R'], max(self.bounds['L'], self.cell - 1))
        if a == 'R': 
            return min(self.bounds['R'], max(self.bounds['L'], self.cell + 1))
        if a == 'U': 
            return min(self.bounds['D'], max(self.bounds['U'], self.cell - 4))
        if a == 'D':
            return min(self.bounds['D'], max(self.bounds['U'], self.cell + 4))
        raise ValueError('Unexpected action')
    

def init_states():
    return [State(i) for i in range(CUBE_SIDE * CUBE_SIDE)]


@dataclass
class Env:
    states: t.List[State] = field(default_factory=init_states)

# The probability of taking the action is 0.25 because we have 4 actions
def policy_evaluation(env, gamma=0.9, theta=1e-2, r=-1):
    """
    Policy evaluation function
    """
    while True:
        delta = 0
        for s in env.states:
            if s.is_termination():
                continue
            v = s.value
            s.value = 0
            for a in s.neighbors:
                s.value += 0.25 * (r + gamma * a)
            delta = max(delta, abs(v - s.value))
        if delta < theta:
            break
        
    return env

def print_states(states):
    for i in range(CUBE_SIDE):
        print([states[i*CUBE_SIDE + j].value for j in range(CUBE_SIDE)])
env = Env()
print("Initial state values")
print_states(env.states)

pol_eval = policy_evaluation(env)

print("\nState values after policy evaluation")
print_states(pol_eval.states)

Initial state values
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

State values after policy evaluation
[0, 0.8, 1.7000000000000002, 2.6, 3.5, 4.175]
[4.625, 5.300000000000001, 6.199999999999999, 7.1, 8.0, 8.675]
[10.025, 10.7, 11.600000000000001, 12.5, 13.4, 14.075000000000003]
[15.425, 16.1, 17.0, 17.9, 18.8, 19.475]
[20.825, 21.5, 22.4, 23.3, 24.2, 24.875]
[25.325000000000003, 26.0, 26.9, 27.8, 28.700000000000003, 0]


La politique gloutonne cherche uniquement à exploiter, sans aucune exploration. A chaque instant, elle choisit l'action qui permet de maximiser la fonction de valeur :

$$\pi(s) = \text{argmax}_a \sum_{s'} p(s'|s,a)[r+\gamma V(s')]$$

**Q2: calculez la politique ainsi obtenue. Vérifiez qu'il s'agit de la politique optimale. Combien d'itérations ont été nécessaires pour obtenir ce résultat ?**

*[Ajoutez votre commentaire ici]*

### II. Algorithme *policy iteration*

Une amélioration de l'algorithme consiste 1) à évaluer la fonction de valeur sur un petit nombre d'itérations (on testera en Q3 avec une seule itération), puis 2) à mettre à jour la politique, puis à recommencer l'étape 1). On peut arrếter l'entraînement lorsque la politique a convergé.

**Q3: implémentez cet algorithme. Est-il plus rapide ?**

*[Ajoutez votre commentaire ici]*

### III. Algorithme *value iteration*

Une autre variante conserve la politique aléatoire tout en long de l'entraînement, mais met à jour la fonction de valeur avec l'équation suivante :

$$v_{k+1} (s) = \max_{a} \sum_{s'} p(s'|s, a)\left[r+\gamma v_k(s') \right]$$

Une fois que la fonction de valeur a convergé, on calcule la politique avec :

$$\pi(s) = argmax_a \sum_{s'} p(s'|s,a)[r+\gamma V(s')]$$


**Q4: implémentez cet algorithme.**

*[Ajoutez votre commentaire ici]*

**Q5: Quel algorithme vous paraît le plus judicieux ?**

*[Ajoutez votre commentaire ici]*